In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
from timm.models.vision_transformer import Block
from EFF3D.res import resnet10

In [ ]:
encoder=resnet10(shortcut_type='B')
encoder=nn.DataParallel(encoder)
# encoder=nn.DataParallel(encoder)

/home/staff/xin/Documents/fastai2/SMRIEFF3D/EFF3D/res.py:174: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [ ]:
weight=torch.load('../nonorm/models/resnet_10_23dataset.pth')

In [ ]:
encoder.load_state_dict(weight['state_dict'])

<All keys matched successfully>

In [ ]:
encoder=encoder.module

In [ ]:
temp=torch.ones(1,1,224,224,224)

In [ ]:
x=encoder(temp)

In [ ]:
x=x.flatten(2).transpose(1, 2)

In [ ]:
x.shape

torch.Size([1, 21952, 512])

In [ ]:
cls_token = nn.Parameter(torch.zeros(1, 1, 512))

In [ ]:
cls_token = cls_token.expand(x.shape[0], -1, -1) 

In [ ]:
x = torch.cat((cls_token, x), dim=1)

In [ ]:
x=torch.ones(1,61,512)

In [ ]:
depth=12
embed_dim=512

In [ ]:
blocks = nn.Sequential(*[
            Block(
                dim=embed_dim, num_heads=4)
            for i in range(depth)])

In [ ]:
x=blocks(x)

In [ ]:
x.shape

torch.Size([1, 61, 512])

In [ ]:
x.shape

In [ ]:
blocks[-1]

Block(
  (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (attn): Attention(
    (qkv): Linear(in_features=512, out_features=1536, bias=False)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=512, out_features=512, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (drop_path): Identity()
  (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (mlp): Mlp(
    (fc1): Linear(in_features=512, out_features=2048, bias=True)
    (act): GELU()
    (fc2): Linear(in_features=2048, out_features=512, bias=True)
    (drop): Dropout(p=0.0, inplace=False)
  )
)